# Basic Neural Net from scratch

This notebook has the purpose of creating a simple neural net from scratch only wiht pytorch. 

To do this, I need: 
 - dependent and independent variable
 - weights & bias initialised
 - linear layer e.g. ReLu
 - activation function: e.g. sigmoid
 - loss function: e.g. mse
 - backwards, i.e. the parameters, gradients


before doing the model - do not forget to clean the data and transform it so that most of the variation is between 0 and 1.

With e.g. divide it by its maximum, so that it becomes like a percentage or use a sigmoid.

## import data and shift it

In [5]:
import pandas as pd

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    df = pd.read_csv("nnbasic-btc-data.csv", index_col=0).reset_index(drop=True)
else:
    df = pd.read_csv("nnbasic-btc-data.csv", index_col=0).reset_index(drop=True)


print(df.shape)
df.head(3)

(11713, 24)


,time_s3,open_s3,high_s3,low_s3,close_s3,vol_s3,time_s2,open_s2,high_s2,low_s2,...,high_s1,low_s1,close_s1,vol_s1,time,open,high,low,close,vol
0,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675,2023.06.01 00:15,27096.9,27096.9,27036.7,...,27077.4,27041.0,27054.9,275.080,2023.06.01 00:45,27054.9,27084.0,27054.8,27084.0,218.143
1,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680,2023.06.01 00:30,27047.0,27077.4,27041.0,...,27084.0,27054.8,27084.0,218.143,2023.06.01 01:00,27084.0,27113.9,27073.5,27100.0,329.412
2,2023.06.01 00:30,27047.0,27077.4,27041.0,27054.9,275.080,2023.06.01 00:45,27054.9,27084.0,27054.8,...,27113.9,27073.5,27100.0,329.412,2023.06.01 01:15,27100.0,27159.0,27100.0,27142.4,979.655


In [ ]:
# do not use the time columns 

df_train = df_merge.filter(items = ['open_s3', 'high_s3', 'low_s3', 'close_s3', 'vol_s3',
       'open_s2', 'high_s2', 'low_s2', 'close_s2', 'vol_s2',
       'open_s1', 'high_s1', 'low_s1', 'close_s1', 'vol_s1', 
       'open', 'high', 'low', 'close', 'vol']).dropna()
print(df_train.shape)
df_train.head(3)

In [ ]:
#df_train
#df_train.filter(items = ["open_s3"])
#df_train.loc[["open_s3"]]



# fastai neural net from scratch

https://www.kaggle.com/code/jhoward/linear-model-and-neural-net-from-scratch

In [ ]:
import torch
from torch import tensor

from https://pytorch.org/tutorials/beginner/nn_tutorial.html

"PyTorch provides methods to create random or zero-filled tensors, which we will use to create our weights and bias for a simple linear model. These are just regular tensors, with one very special addition: we tell PyTorch that they require a gradient. **This causes PyTorch to record all of the operations done on the tensor, so that it can calculate the gradient during back-propagation automatically!**

For the weights, we set requires_grad after the initialization, since we don’t want that step included in the gradient. (Note that a trailing _ in PyTorch signifies that the operation is performed in-place.)"

## dep, indep & weights

In this setting we will just try to predict the high value based on the previous three candles. The results will likely not be good since point predictions are tricky.

In [ ]:
t_dep = tensor(df_train['high'].values, dtype=torch.float)
print(t_dep.shape) # 1 D tensor
t_dep

All variables are used instead of the high value which is the dependend value.

In [ ]:
t_indep = tensor(df_train.loc[:, df_train.columns != "high"].values, dtype=torch.float)

t_indep.shape # 2 D tensor > rows with observations and columns with features

The number of features needs to be the same as the lenght of the weights, because they will be multiplicated.

In [ ]:
import math

n,c = t_indep.shape

# "We are initializing the weights here with Xavier initialisation (by multiplying with 1/sqrt(n))."
weights = torch.randn(c) / math.sqrt(n)
weights.requires_grad_()
bias = torch.zeros(n, requires_grad=True)
weights.shape

In [ ]:
t_indep*weights

### Sidebar: torch.randn
short check of the torch.randn function: https://pytorch.org/docs/stable/generated/torch.randn.html#torch.randn

In [ ]:
torch.randn(19, 10)

In [ ]:

# coefficients = weights
# make them from -0.5 to 0.5 from a normal distribution
coeffs = torch.rand( t_indep.shape[1] )-0.5

print(coeffs.shape)
coeffs

In [ ]:
(t_indep*coeffs)[0]

As you can see, there are a lot of numbers up and down, since the prices range roughly from 30k to about 60k. 

Note to myself: if it does not interfere with cross entropy then I should use log on all prices.
cross entropy uses log in its calculations and it depends on the implementation if it does interfere (I guess right now).

## making the first prediction

Since we are making a linear classifier the formula for yhat or the predictions is the sum of the products of coefficients and observations. 

In other words: 

$ y = x_0 \times b_0 + x_1 \times b_1 $

where $x_0$ is the first vector of observations (=features) and $b_0$ is the first coefficient.

and then is $y$ the vector with the predictions the same length as the data frame, so this formula can be read "rowwise".

In [ ]:
preds = (t_indep*coeffs).sum(axis=1)
print(preds.shape) # 1d vector of predictions
preds[0]

In [ ]:
# quick check for first row: 

print(t_indep[0])
print(coeffs)
print("\n first entry of Indep * coeffs: \n",t_indep[0][0]*coeffs[0])
print("\n Indep * coeffs: \n",t_indep[0]*coeffs)
print("\n sum of Indep * coeffs: \n", (t_indep[0]*coeffs).sum() ) #axis=1 is along the colums


## loss: cross entropy 

In [ ]:
import torch.nn.functional as F
loss_ce = F.cross_entropy(preds, t_dep)
loss_mae = F.l1_loss(preds, t_dep)
loss_mae = F.mse_loss(preds, t_dep)

print("mean absolute error:",loss_mae)
print("mean squared error:",loss_mae)
print("cross entropy:",loss_ce)

## gradients & learning rate

In [ ]:
coeffs

In [ ]:
coeffs.requires_grad_()

In [ ]:
# I am not sure if the above way works if I only use the loss functions in a functional style
# therefore, I document a way to use it instantiated as a class

from torch import nn

cel = nn.CrossEntropyLoss()

preds = (t_indep*coeffs.requires_grad_()).sum(axis=1)

loss = cel(preds,t_dep)
print(loss)
loss.backward(retain_graph=True)

gradients = coeffs.grad
print(gradients)

In [ ]:
loss.backward(retain_graph=True)
print(coeffs.grad)

In [ ]:
# initialise coeffs and define the independent variable 
t_indep*coeffs

# define it as a function 
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
    
preds = (t_indep*coeffs).sum(axis=1)

In [ ]:
# define the loss function

# herer it is mean absolute error
loss = torch.abs(preds-t_dep).mean()

# define it in a function
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

## next steps

 - figure out when and how to use backward() and require_grad
 - put the pieces togehter: calc preds > calc loss > get gradients > subtract gradients*learningrate with preds

Note on gradients: \n
My current understand is that you need to specify the tensor of where to calculate the gradients on, to access them explicitly. \n
It seems, that the backward function needs to know the input, target and weights.

# pytorch beginner tutorial by jeremy howard

https://pytorch.org/tutorials/beginner/nn_tutorial.html

In [ ]:
import torch
xb = torch.randn(64, 784)
weights = torch.randn(784, 10)

In [ ]:
pred = xb@weights
pred.shape

In [ ]:
print(xb.shape)
print(weights.shape)

# pytorch basics tutorial
see: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [ ]:
import torch
from torch import nn

# define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [ ]:
model = NeuralNetwork()
print(model)

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [ ]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

### Sidebar: nn.linear

In [ ]:
input_image = torch.ones(3,2,2)
print(input_image.size())

input_image[1].add_(1)
input_image[2].add_(2)
input_image[0,1,1].add_(1)

input_image

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
flat_image

In [ ]:

layer1 = nn.Linear(in_features=2*2, out_features=2,bias=False)
import torch.nn.init as init
init.ones_(layer1.weight)

linear_layer = layer1(flat_image)

print(f"\nlinear layer shape:{linear_layer.size()}" )
w = layer1.weight # has the in_features shape, meaning it is a vector of length 4
print(f"\nweight shape:{w.shape} = [out_features,in_features]")
print("\nlinear layer weights:\n",w)
#print("\noutput divided by input\n",linear_layer/flat_image)
print(f"\ny=output = \n{linear_layer}")
# output should be: 
flat_image@w.T

There for the transformation of $ y = xW^T+b $ where x=in_features, y=out_features, W=weights and b=bias

 - nn.Linear generates a random weight matrix with shape of [out_features,in_features]
 - nn.Linear adds all the columns together
 - the added rows are multiplied with the weight matrix
 - optionally a bias term is added

By summing all input columns a matrix with length of in_features is present to be multiplied with the correct dimension of the weight matrix.

see: https://ashwinhprasad.medium.com/pytorch-for-deep-learning-nn-linear-and-nn-relu-explained-77f3e1007dbb

## sidebar: tensors

One can think of tensors in terms of the below element has an element of 3 which contains 2 elements which contain 4 elements which contain 2 elements, hence resultig in a nested list of two.

see also: 
 - https://stackoverflow.com/questions/52370008/understanding-pytorch-tensor-shape
 - https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

In [ ]:
test = torch.randn(3,2,4,2)
test

In [ ]:
#the first layer of the tensor consists of three elements of which the third is printed
# this third element of the first layer has 2 elements which contain 4 elements which in turn contain 2
# the last two form then the vector as printed in the brackets
test[2] 

## next